# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("../output_data/cities_550.csv")
del df["Unnamed: 0"]
df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED
0,"{'coord': {'lon': -175.2, 'lat': -21.2}, 'weat...",2022-07-31 02:58:16,Vaini,TO,-21.2000,-175.2000,80.76,89.0,40.0,10.36
1,"{'coord': {'lon': -122.4286, 'lat': 37.4636}, ...",2022-07-31 02:58:18,Half Moon Bay,US,37.4636,-122.4286,62.33,87.0,100.0,16.11
2,"{'coord': {'lon': 168.3333, 'lat': -46.6}, 'we...",2022-07-31 02:58:19,Bluff,NZ,-46.6000,168.3333,47.80,77.0,49.0,16.26
3,"{'coord': {'lon': -79.2353, 'lat': 42.097}, 'w...",2022-07-31 02:58:21,Jamestown,US,42.0970,-79.2353,61.48,55.0,1.0,1.63
4,"{'coord': {'lon': -73.9662, 'lat': 42.6001}, '...",2022-07-31 02:57:37,Albany,US,42.6001,-73.9662,65.89,78.0,0.0,4.85
...,...,...,...,...,...,...,...,...,...,...
498,"{'coord': {'lon': -157.8378, 'lat': 21.4447}, ...",2022-07-31 03:15:04,‘Āhuimanu,US,21.4447,-157.8378,81.55,71.0,75.0,16.11
499,"{'coord': {'lon': 36.8595, 'lat': 49.4627}, 'w...",2022-07-31 03:15:06,Balakliya,UA,49.4627,36.8595,60.71,71.0,0.0,6.76
500,"{'coord': {'lon': -60.2177, 'lat': 46.2367}, '...",2022-07-31 03:15:09,Sydney Mines,CA,46.2367,-60.2177,64.26,94.0,75.0,10.36
501,"{'coord': {'lon': 44.8414, 'lat': 51.5506}, 'w...",2022-07-31 03:15:11,Lysyye Gory,RU,51.5506,44.8414,65.25,86.0,100.0,9.40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key) 

In [5]:
locations = df[["LAT", "LNG"]]

weights = df["HUMIDITY"]

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [7]:
best_cities_df = df.loc[ (df["MAX_TEMP"].between(69, 84)) & (df["HUMIDITY"] < 80) & (df["CLOUDINESS"] < 50), :]
best_cities_df.dropna()
best_cities_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED
10,"{'coord': {'lon': -114.0611, 'lat': 27.9769}, ...",2022-07-31 02:58:37,Guerrero Negro,MX,27.9769,-114.0611,70.70,77.0,5.0,13.85
35,"{'coord': {'lon': -134.9692, 'lat': -23.1203},...",2022-07-31 02:59:30,Rikitea,PF,-23.1203,-134.9692,72.93,75.0,9.0,15.93
37,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43
41,"{'coord': {'lon': -159.775, 'lat': -21.2078}, ...",2022-07-31 02:59:41,Avarua,CK,-21.2078,-159.7750,78.85,69.0,40.0,20.71
48,"{'coord': {'lon': -17.1, 'lat': 32.6667}, 'wea...",2022-07-31 02:59:55,Ponta do Sol,PT,32.6667,-17.1000,69.94,79.0,14.0,6.98
...,...,...,...,...,...,...,...,...,...,...
458,"{'coord': {'lon': -108.1833, 'lat': 25.3667}, ...",2022-07-31 03:13:42,Angostura,MX,25.3667,-108.1833,82.00,72.0,10.0,3.76
464,"{'coord': {'lon': 118.7833, 'lat': 42.1167}, '...",2022-07-31 03:13:54,Mujiayingzi,CN,42.1167,118.7833,83.73,50.0,15.0,6.82
466,"{'coord': {'lon': 17.1369, 'lat': 42.9622}, 'w...",2022-07-31 03:13:58,Korčula,HR,42.9622,17.1369,76.62,68.0,0.0,7.02
470,"{'coord': {'lon': 10.85, 'lat': 30.3333}, 'wea...",2022-07-31 03:14:06,Nālūt,LY,30.3333,10.8500,78.46,28.0,0.0,10.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = best_cities_df.sample(10)

hotel_df["HOTEL_PLACEID"] = "NULL"
hotel_df["HOTEL_NAME"] = "NULL"
hotel_df["HOTEL_CITY"] = "NULL"
hotel_df["HOTEL_COUNTRY"] = "NULL"

hotel_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
37,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43,NULL,NULL,NULL,NULL
309,"{'coord': {'lon': 55.2, 'lat': 45.3167}, 'weat...",2022-07-31 03:08:34,Beyneu,KZ,45.3167,55.2000,83.57,36.0,28.0,5.61,NULL,NULL,NULL,NULL
404,"{'coord': {'lon': 9.2036, 'lat': 39.3042}, 'we...",2022-07-31 03:11:44,Sinnai,IT,39.3042,9.2036,73.47,65.0,0.0,9.22,NULL,NULL,NULL,NULL
202,"{'coord': {'lon': 96.37, 'lat': 61.67}, 'weath...",2022-07-31 03:04:53,Baykit,RU,61.6700,96.3700,75.83,69.0,39.0,4.79,NULL,NULL,NULL,NULL
414,"{'coord': {'lon': -56.0861, 'lat': -9.8756}, '...",2022-07-31 03:12:09,Alta Floresta,BR,-9.8756,-56.0861,70.54,41.0,0.0,2.91,NULL,NULL,NULL,NULL
433,"{'coord': {'lon': 10.509, 'lat': 36.0303}, 'we...",2022-07-31 03:12:49,Hergla,TN,36.0303,10.5090,76.86,61.0,0.0,4.61,NULL,NULL,NULL,NULL
136,"{'coord': {'lon': -28.7, 'lat': 38.5167}, 'wea...",2022-07-31 03:02:48,Ribeira Grande,PT,38.5167,-28.7000,73.56,71.0,17.0,2.48,NULL,NULL,NULL,NULL
280,"{'coord': {'lon': 119.1594, 'lat': 34.5997}, '...",2022-07-31 03:07:30,Xinpu,CN,34.5997,119.1594,83.71,77.0,42.0,12.59,NULL,NULL,NULL,NULL
466,"{'coord': {'lon': 17.1369, 'lat': 42.9622}, 'w...",2022-07-31 03:13:58,Korčula,HR,42.9622,17.1369,76.62,68.0,0.0,7.02,NULL,NULL,NULL,NULL
413,"{'coord': {'lon': -8.3327, 'lat': 40.1629}, 'w...",2022-07-31 03:12:05,Lata,PT,40.1629,-8.3327,69.71,56.0,0.0,3.83,NULL,NULL,NULL,NULL


In [10]:
# params dictionary to update each iteration

params = {
    "location": "0,0",
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    
    lat = row["LAT"]
    lng = row["LNG"]

    # change location each iteration while leaving original params in place
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_data = requests.get(base_url, params=params)

    # convert to json
    hotel_json = hotel_data.json()
    # pprint(hotel_json)

#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.

    try:
        
        hotel_df.loc[index, "HOTEL_PLACEID"] = hotel_json["results"][0]["place_id"]
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [15]:
cleanhotel_df = hotel_df.loc[ (hotel_df["HOTEL_PLACEID"] != "NULL"), :]

cleanhotel_df = cleanhotel_df.reset_index(drop=True)

cleanhotel_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
0,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43,ChIJL4z7-PMhN3YRhdXuLcxVEH4,NULL,NULL,NULL
1,"{'coord': {'lon': 55.2, 'lat': 45.3167}, 'weat...",2022-07-31 03:08:34,Beyneu,KZ,45.3167,55.2000,83.57,36.0,28.0,5.61,ChIJJRMpDd71vkERIXKQYzQ2Xqk,NULL,NULL,NULL
2,"{'coord': {'lon': 9.2036, 'lat': 39.3042}, 'we...",2022-07-31 03:11:44,Sinnai,IT,39.3042,9.2036,73.47,65.0,0.0,9.22,ChIJD5MTrUe14BIRBtJa6v9pXcE,NULL,NULL,NULL
3,"{'coord': {'lon': -56.0861, 'lat': -9.8756}, '...",2022-07-31 03:12:09,Alta Floresta,BR,-9.8756,-56.0861,70.54,41.0,0.0,2.91,ChIJrXCtD6ExqpMRsmS2tyRDVOo,NULL,NULL,NULL
4,"{'coord': {'lon': 10.509, 'lat': 36.0303}, 'we...",2022-07-31 03:12:49,Hergla,TN,36.0303,10.5090,76.86,61.0,0.0,4.61,ChIJv9StbjqE_RIRC0BwiRrGSmI,NULL,NULL,NULL
5,"{'coord': {'lon': -28.7, 'lat': 38.5167}, 'wea...",2022-07-31 03:02:48,Ribeira Grande,PT,38.5167,-28.7000,73.56,71.0,17.0,2.48,ChIJ02FfGFl1OAsR_4icUj4oDxY,NULL,NULL,NULL
6,"{'coord': {'lon': 119.1594, 'lat': 34.5997}, '...",2022-07-31 03:07:30,Xinpu,CN,34.5997,119.1594,83.71,77.0,42.0,12.59,ChIJT_21ytTpuDURK_ePHn6i0p0,NULL,NULL,NULL
7,"{'coord': {'lon': 17.1369, 'lat': 42.9622}, 'w...",2022-07-31 03:13:58,Korčula,HR,42.9622,17.1369,76.62,68.0,0.0,7.02,ChIJJy6mD7RTShMRXk304DJOb6w,NULL,NULL,NULL
8,"{'coord': {'lon': -8.3327, 'lat': 40.1629}, 'w...",2022-07-31 03:12:05,Lata,PT,40.1629,-8.3327,69.71,56.0,0.0,3.83,ChIJ94bq38PvIg0RK-4myM-1Pu0,NULL,NULL,NULL


In [28]:
# params dictionary to update each iteration

params = {
    "place_id": "",
    "fields": "name,formatted_address",
    "key": g_key
}


for index, row in cleanhotel_df.iterrows():
    
    place_id = row["HOTEL_PLACEID"]

    # change location each iteration while leaving original params in place
    
    params["place_id"] = f"{place_id}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/details/json?"

    # make request and print url
    address_data = requests.get(base_url, params=params)

    # convert to json
    address_json = address_data.json()
    
    # pprint(address_json)
    
    try:
        
        cleanhotel_df.loc[index, "HOTEL_NAME"] = address_json["result"]["name"]
        
        cleanhotel_df.loc[index, "HOTEL_CITY"] = address_json["result"]["formatted_address"].split(",")[-2].strip()
    
        cleanhotel_df.loc[index, "HOTEL_COUNTRY"] = address_json["result"]["formatted_address"].split(",")[-1].strip()
    
    except:
        
        print("Missing field/result... skipping.")    


In [29]:
cleanhotel_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
0,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43,ChIJL4z7-PMhN3YRhdXuLcxVEH4,Hôtel Hiva Oa Hanakee Lodge,Hiva Oa,French Polynesia
1,"{'coord': {'lon': 55.2, 'lat': 45.3167}, 'weat...",2022-07-31 03:08:34,Beyneu,KZ,45.3167,55.2000,83.57,36.0,28.0,5.61,ChIJJRMpDd71vkERIXKQYzQ2Xqk,Gostinitsa Zhibek-Zholy Beyneu,Beyneu 130100,Kazakhstan
2,"{'coord': {'lon': 9.2036, 'lat': 39.3042}, 'we...",2022-07-31 03:11:44,Sinnai,IT,39.3042,9.2036,73.47,65.0,0.0,9.22,ChIJD5MTrUe14BIRBtJa6v9pXcE,White House & B.,09048 Sinnai CA,Italy
3,"{'coord': {'lon': -56.0861, 'lat': -9.8756}, '...",2022-07-31 03:12:09,Alta Floresta,BR,-9.8756,-56.0861,70.54,41.0,0.0,2.91,ChIJrXCtD6ExqpMRsmS2tyRDVOo,Lisboa Palace Hotel,78580-000,Brazil
4,"{'coord': {'lon': 10.509, 'lat': 36.0303}, 'we...",2022-07-31 03:12:49,Hergla,TN,36.0303,10.5090,76.86,61.0,0.0,4.61,ChIJv9StbjqE_RIRC0BwiRrGSmI,Dar Hergla,La Falaise Hergla TN 4012، Hergla,Tunisia
5,"{'coord': {'lon': -28.7, 'lat': 38.5167}, 'wea...",2022-07-31 03:02:48,Ribeira Grande,PT,38.5167,-28.7000,73.56,71.0,17.0,2.48,ChIJ02FfGFl1OAsR_4icUj4oDxY,Casa da Japoneira,9900-361 Horta,Portugal
6,"{'coord': {'lon': 119.1594, 'lat': 34.5997}, '...",2022-07-31 03:07:30,Xinpu,CN,34.5997,119.1594,83.71,77.0,42.0,12.59,ChIJT_21ytTpuDURK_ePHn6i0p0,Sofitel,Hai Zhou Qu,海连中路7号 邮政编码: 222000
7,"{'coord': {'lon': 17.1369, 'lat': 42.9622}, 'w...",2022-07-31 03:13:58,Korčula,HR,42.9622,17.1369,76.62,68.0,0.0,7.02,ChIJJy6mD7RTShMRXk304DJOb6w,"Heritage hotel ""Adriatic"" Adults only",Orebić,Croatia
8,"{'coord': {'lon': -8.3327, 'lat': 40.1629}, 'w...",2022-07-31 03:12:05,Lata,PT,40.1629,-8.3327,69.71,56.0,0.0,3.83,ChIJ94bq38PvIg0RK-4myM-1Pu0,Magnolias Cottage,3200-048 Lousã,Portugal


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{HOTEL_NAME}</dd>
<dt>City</dt><dd>{HOTEL_CITY}</dd>
<dt>Country</dt><dd>{HOTEL_COUNTRY}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in cleanhotel_df.iterrows()]
locations = cleanhotel_df[["LAT", "LNG"]]

In [33]:
# Add marker layer ontop of heat map

 # Assign the marker layer to a variable
    
markers = gmaps.marker_layer(locations=locations, display_info_box=True, info_box_content=hotel_info)

# Add the layer to the map

fig.add_layer(markers)


fig




Figure(layout=FigureLayout(height='420px'))